In [3]:
import pandas as pd
import numpy as np

In [4]:
FH=pd.read_csv('./data/CleanedPlayerData/MergedHitters.csv')
FP=pd.read_csv('./data/CleanedPlayerData/MergedPitchers.csv')

In [ ]:
pctcols=[i for i in FH.columns if '%' in i]
for i in pctcols:
    FH[i]=FH[i].str.replace('%','').astype(float)/100
pctcols=[i for i in FP.columns if '%' in i]
for i in pctcols:
    FP[i]=FP[i].str.replace('%','').astype(float)/100
FH['hHR/FB']=FH['hHR/FB'].str.replace('%','').astype(float)/100
FP['pHR/FB']=FP['pHR/FB'].str.replace('%','').astype(float)/100

In [6]:
FH.columns

Index(['hName', 'hTeam', 'hPA', 'hBB%', 'hK%', 'hBB/K', 'hAVG', 'hOBP', 'hSLG',
       'hOPS', 'hISO', 'hSpd', 'hBABIP', 'hUBR', 'hwGDP', 'hwSB', 'hwRC',
       'hwRAA', 'hwOBA', 'hwRC+', 'hplayerid', 'hGB/FB', 'hLD%', 'hGB%',
       'hFB%', 'hIFFB%', 'hHR/FB', 'hIFH', 'hIFH%', 'hBUH', 'hBUH%', 'hPull%',
       'hCent%', 'hOppo%', 'hSoft%', 'hMed%', 'hHard%', 'hG', 'hAB', 'hH',
       'h1B', 'h2B', 'h3B', 'hHR', 'hR', 'hRBI', 'hBB', 'hIBB', 'hSO', 'hHBP',
       'hSF', 'hSH', 'hGDP', 'hSB', 'hCS', 'hseason', 'hlast_name',
       'h first_name', 'hplayer_id', 'hattempts', 'havg_hit_angle',
       'hanglesweetspotpercent', 'hmax_hit_speed', 'havg_hit_speed', 'hfbld',
       'hgb', 'hmax_distance', 'havg_distance', 'havg_hr_distance',
       'hev95plus', 'hev95percent', 'hbarrels', 'hbrl_percent', 'hbrl_pa'],
      dtype='object')

In [7]:
FP.columns

Index(['pName', 'pTeam', 'pK/9', 'pBB/9', 'pK/BB', 'pHR/9', 'pK%', 'pBB%',
       'pK-BB%', 'pAVG', 'pWHIP', 'pBABIP', 'pLOB%', 'pERA-', 'pFIP-',
       'pxFIP-', 'pERA', 'pFIP', 'pE-F', 'pxFIP', 'pSIERA', 'pplayerid',
       'pGB/FB', 'pLD%', 'pGB%', 'pFB%', 'pIFFB%', 'pHR/FB', 'pRS', 'pRS/9',
       'pBalls', 'pStrikes', 'pPitches', 'pPull%', 'pCent%', 'pOppo%',
       'pSoft%', 'pMed%', 'pHard%', 'pW', 'pL', 'pG', 'pGS', 'pCG', 'pShO',
       'pSV', 'pHLD', 'pBS', 'pIP', 'pTBF', 'pH', 'pR', 'pER', 'pHR', 'pBB',
       'pIBB', 'pHBP', 'pWP', 'pBK', 'pSO', 'pseason', 'plast_name',
       'p first_name', 'pplayer_id', 'pattempts', 'pavg_hit_angle',
       'panglesweetspotpercent', 'pmax_hit_speed', 'pavg_hit_speed', 'pfbld',
       'pgb', 'pmax_distance', 'pavg_distance', 'pavg_hr_distance',
       'pev95plus', 'pev95percent', 'pbarrels', 'pbrl_percent', 'pbrl_pa'],
      dtype='object')

In [10]:
def construct_input(atts,parr,harr):
    atts[2]=float(atts[2])
    features=['stand','p_throws', 'season',
          'hBB%','hK%','hSpd','hPull%','hGB%','hLD%','hFB%',
          'havg_hit_angle','hmax_hit_speed', 'havg_hit_speed', 'hfbld','hgb','hev95percent',
          'pBB%','pK%','pPull%','pGB%','pLD%','pFB%',
          'pavg_hit_angle','pmax_hit_speed', 'pavg_hit_speed', 'pfbld','pgb','pev95percent']
    hfeats=['hBB%','hK%','hSpd','hPull%','hGB%','hLD%','hFB%',
          'havg_hit_angle','hmax_hit_speed', 'havg_hit_speed', 'hfbld','hgb','hev95percent']
    pfeats=['pBB%','pK%','pPull%','pGB%','pLD%','pFB%',
          'pavg_hit_angle','pmax_hit_speed', 'pavg_hit_speed', 'pfbld','pgb','pev95percent']
    data_z=np.zeros((1,3))
    df=pd.DataFrame(data=data_z,columns=features[0:3])
    df.loc[0,'stand']=atts[0]
    df.loc[0,'p_throws']=atts[1]
    df.loc[0,'season']=atts[2]
    for i in hfeats:
        df[i]=harr[i].values
    for i in pfeats:
        df[i]=parr[i].values
    return df

In [19]:
def simAB(pitcher,hitter,ispRH=True,isbRH=True,pseason=2019,hseason=2019,season=2019,output='probs'):
    pitcher=FP[(FP['pName']==pitcher)&(FP['pseason']==pseason)]
    hitter=FH[(FH['hName']==hitter)&(FH['hseason']==hseason)]
    atts=[float(int(isbRH)),float(int(ispRH)),season]
    feats_arr=construct_input(atts,pitcher,hitter)
    feats_sc=ss.transform(feats_arr)
    preds=model.predict(feats_sc)
    preds_clean=np.round(preds,3)
    if output=='probs':
        cols=['GB', '2B', '1B', 'K', 'BB', 'PU', 'FB', 'LD', 'HR', 'HBP', '3B']
        df=pd.DataFrame(data=preds_clean,columns=cols)
    return df

In [20]:
simAB('Corey Kluber','Miguel Sano')

NameError: name 'ss' is not defined